In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Homework 7 by Ahmed In this data analysis covid data and population data across Germany with respect to gender states and age groups will be put into context. First we will gain a rough understanding over the indivudual data sets. Further down we will see some insights based on mathematical and compositional transformed data. 

# Loading libraries and data

In [ ]:
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
import pandas as pd
dem_df = pd.read_csv('../input/covid19-tracking-germany/demographics_de.csv')
cov_df = pd.read_csv('../input/covid19-tracking-germany/covid_de.csv', parse_dates=True)

# First overview of data

In [ ]:
dem_df.head()

In [ ]:
cov_df.head()

Deducing some constants out of data

In [ ]:
total_population = dem_df.population.sum()
total_fem_population = dem_df.loc[dem_df.gender == 'female'].population.sum()
total_men_population = dem_df.loc[dem_df.gender == 'male'].population.sum()

print('total populationof Germany:' , total_population)
print('total female population of Germany: ', total_fem_population)
print('total male population of Germany: ' , total_men_population)

#Change datatype of Date values
cov_df.date= pd.to_datetime(cov_df.date)

print('first date: ', cov_df.date.min())
print('last date: ', cov_df.date.max())

print('size demography data: ', dem_df.shape )
print('size covid data: ', cov_df.shape )


# **Check how continuous Datacollection is:**

In [ ]:
date_groups = cov_df.copy()

date_groups.date.value_counts().plot();
plt.title("Number of data collected per day")
plt.xlabel("Date")

In [ ]:
cov_case_time = cov_df.groupby('date').cases.sum().to_frame()
cov_deaths_time = cov_df.groupby('date').deaths.sum().to_frame()

cov_cases_deaths_time = cov_case_time.join(cov_deaths_time)

cov_cases_deaths_time.plot();
plt.title("Cases and deaths caused by covid in Germany")
plt.xlabel("Date")

cov_cases_deaths_time.head(2)

In [ ]:
#cov_df.reset_index()
#cov_df.set_index('date', inplace=True)
#cov_df.head(3)

# preperation for Join

In [ ]:
di = {'female': "F", 'male': "M"}
dem_df.replace({"gender": di}, inplace= True)
dem_df.head(3)

In [ ]:
cov_df['unique_id'] = cov_df.state.astype(str) + '_' + cov_df.age_group.astype(str) + '_'+ cov_df.gender.astype(str)
cov_df.reset_index()
cov_df.head(2)

In [ ]:
dem_df['unique_id'] = dem_df.state.astype(str) + '_' + dem_df.age_group.astype(str) + '_'+ dem_df.gender.astype(str)
dem_df.head(2)

In [ ]:
cov_dem_df = cov_df.merge(dem_df, on=['unique_id'])
cov_dem_df.head(3)


In [ ]:
cov_dem_df.reset_index()
cov_dem_df.set_index('date', inplace=True)
cov_dem_df.head(3)

In [ ]:
cov_dem_df.drop(columns=['unique_id', 'state_y', 'age_group_y', 'gender_y'], inplace = True)
cov_dem_df.head(3)

In [ ]:
df = pd.DataFrame({'lab':['A', 'B', 'C'], 
                   'val':[10, 30, 20]})
df

In [ ]:
fem = cov_dem_df.loc[cov_dem_df.gender_x == 'F'].cases.sum()/total_population*100
men = cov_dem_df.loc[cov_dem_df.gender_x == 'M'].cases.sum()/total_population*100

gender_split_df = pd.DataFrame({'Cases': [fem, men],
                                'Gender':['FEM', 'MEN']})

#f_cases_ger = cov_dem_df.loc[cov_dem_df.gender_x == 'F'].cases.sum()
#m_cases_ger = cov_dem_df.loc[cov_dem_df.gender_x == 'M'].cases.sum()

ax = gender_split_df.plot.bar(x='Gender', y='Cases', rot=0)
plt.title("Covid cases relativ to total german population")
plt.xlabel("Gender")
plt.ylabel("%")

gender_split_df

In [ ]:
fem = cov_dem_df.loc[cov_dem_df.gender_x == 'F'].cases.sum()/total_fem_population*100
men = cov_dem_df.loc[cov_dem_df.gender_x == 'M'].cases.sum()/total_men_population*100

gender_split_df = pd.DataFrame({'Cases': [fem, men],
                                'Gender':['FEM', 'MEN']})

ax = gender_split_df.plot.bar(x='Gender', y='Cases', rot=0)
plt.title("Covid cases relativ to total same sex population")
plt.xlabel("Gender")
plt.ylabel("%")

gender_split_df

In [ ]:
fem_df = cov_dem_df.loc[cov_dem_df.gender_x == 'F']
fem_gender_cases_df = fem_df.groupby('age_group_x').cases.sum().to_frame()# ,as_index=False
fem_gender_popul_df = dem_df.loc[dem_df.gender == 'F'].groupby('age_group').population.sum().to_frame()
fem_join_df = fem_gender_cases_df.join(fem_gender_popul_df)

#population is never 0 or negative
fem_join_df['fem_rel_cases'] = fem_join_df.cases/fem_join_df.population *100
fem_join_df.drop(columns=['cases', 'population'], inplace = True)

men_df = cov_dem_df.loc[cov_dem_df.gender_x == 'M']
men_gender_cases_df = men_df.groupby('age_group_x').cases.sum().to_frame()# ,as_index=False
men_gender_popul_df = dem_df.loc[dem_df.gender == 'M'].groupby('age_group').population.sum().to_frame()
men_join_df = men_gender_cases_df.join(men_gender_popul_df)

#population is never 0 or negative
men_join_df['men_rel_cases'] = men_join_df.cases/men_join_df.population *100
men_join_df.drop(columns=['cases', 'population'], inplace = True)

ages_gender_df = fem_join_df.join(men_join_df)
ages_gender_df.columns = ['Fem', 'Men']

ax = ages_gender_df.plot.bar(rot=0)
plt.title("Relative covid cases among of same sex and age group in Germany")
plt.xlabel("Age groups")
plt.ylabel("%")

ages_gender_df.head(3)

In [ ]:
men_gender_popul_df.columns=['Men_population']
fem_gender_popul_df.columns=['Fem_population']

population_destribution = fem_gender_popul_df.join(men_gender_popul_df)
population_destribution['total'] = (population_destribution.Men_population + population_destribution.Fem_population)/ total_population *10
population_destribution.drop(columns=['Men_population', 'Fem_population'], inplace = True)

population_destribution.columns = ['rel. share of age group *1/10']
#ax = population_destribution.plot()
ax = population_destribution.plot(color='green', linestyle='dashed', label='share of age group on total population' )
ages_gender_df.plot.bar(rot=0,ax = ax)


plt.ylabel("%")
plt.title("Number of data collected per day")
plt.xlabel("Date")

#.plot(x='month', kind='bar',
#    , ax = ax                                                    ax=ax)


relativ_population_destribution =  population_destribution
population_destribution.head()
population_destribution.head()

In [ ]:
state_cases_df = cov_dem_df.groupby('state_x').cases.sum().to_frame()# ,as_index=False
cov_dem_df.head(3)

#state_dem = 
state_cases_popul_df = state_cases_df.join( dem_df.groupby('state').population.sum().to_frame() )
state_cases_popul_df['rel_cases'] = state_cases_popul_df.cases / state_cases_popul_df.population *100
state_cases_popul_df.sort_values(by='rel_cases',  ascending=False,inplace = True)
state_cases_popul_df.head()

In [ ]:
#The Column name is removed just for better depiction
state_cases_popul_df.rename({'rel_cases': ' '}, axis=1, inplace=True)
plot = state_cases_popul_df.plot.pie(y=' ',  figsize=(5, 5))
plot.get_legend().remove()
plot.set_title('Cases relative to size of local population', pad=20)

In [ ]:
2.4*1.1

In [ ]:
speed = [0.1, 17.5, 40, 48, 52, 69, 88]

lifespen = [2, 8, 70, 1.5, 25, 12, 28]

index = ['snail', 'pig', 'elephant',

         'rabbit', 'giraffe', 'coyote', 'horse']

df = pd.DataFrame({'speed': speed,

                   'lifespan': lifespan}, index=index)

ax = df.plot.bar(rot=0)
df.head(3)

In [ ]:
df = pd.DataFrame({'lab':['A', 'B', 'C'], 'val':[10, 30, 20]})

ax = df.plot.bar(x='lab', y='val', rot=0)


In [ ]:
sns.barplot(date_groups.index, date_groups.values)


g = sns.barplot(x=tmp_valc.index, y=tmp_valc)
t = g.set(title="Value counts of Pandas Series")

In [ ]:
dem_df.head(3)

In [ ]:
multi_cov = cov_df.groupby(['state', 'gender','age_group']).cases.agg(sum)
multi_cov

In [ ]:
multi_dem = dem_df.groupby(['state', 'gender','age_group'])
multi_dem.head(3)

In [ ]:
cov_df.join(multi_dem)

In [ ]:
plt.title("Number of data collected per day")
sns.barplot(x = sorted(cov_df.date.unique()),data=time_groups)
plt.xlabel("Date")


Obviously not for each date the same  values were collected.

1. # Time Range of given time periode

In [ ]:
min = cov_df.date.unique().min()
max = cov_df.date.unique().max()
print( min, max)

In [ ]:


plt.figure(figsize=(7,3))
plt.title("Number of data collected per day")
sns.barplot(x = 'date', y=time_groups)
plt.xlabel("Date")

In [ ]:
prob_groups = cov_df.date.value_counts()
prob_groups = cov_df.groupby('date').sum()
plt.figure(figsize=(16,6))
prob_groups.plot(kind='bar')
plt.xticks(rotation=0)


plt.show()

In [ ]:
age_group_df =cov_df.groupby(['state', 'age_group', 'gender'])
age_group_df.head(3)